In [1]:
from weiner_variation.sim.data_structures import DrawInput
from weiner_variation.sim.process import PASS_SEQUENCE, DIAMETER, TEMPERATURE, create_in_profile, DIAMETER_STD, TEMPERATURE_STD
from weiner_variation.sim.config import SAMPLE_COUNT, FIELDS, SEED
from weiner_variation.config import DATA_DIR

import pandas as pd
import numpy as np
from scipy.stats import norm
from copy import deepcopy
import tqdm
from multiprocessing import Pool
import os

/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:79: RuntimeWarning: divide by zero encountered in scalar divide
  _r2 = (depth - l23(_alpha) * np.sin(_alpha) - fh(_alpha)) / (1 - np.cos(_alpha))
/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:83: RuntimeWarning: invalid value encountered in scalar multiply
  width / 2 - _r2 * np.sin(_alpha) - l23(_alpha) * np.cos(_alpha)
/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:84: RuntimeWarning: invalid value encountered in scalar multiply
  - fw(_alpha) - (_r2 + r4) * np.sin(_alpha4)
/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:79: RuntimeWarning: divide by zero encountered in scalar divide
  _r2 = (de

In [2]:
import pyroll.basic as pr

In [3]:
DIAMETER_STD = DIAMETER_STD
TEMPERATURE_STD = TEMPERATURE_STD
OUTPUT_FILENAME = "sim_input_results.csv"

In [4]:
diameter_dist = norm(loc=DIAMETER, scale=DIAMETER_STD)
temperature_dist = norm(loc=TEMPERATURE, scale=TEMPERATURE_STD)

In [5]:
RNG = np.random.default_rng(SEED)
diameters = diameter_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
temperatures = temperature_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)

draws = [
    DrawInput(d, t)
    for d, t in zip(diameters, temperatures)
]

In [6]:
def worker(draw: DrawInput):    
    ip = create_in_profile(
            diameter=draw.diameter,
            temperature=draw.temperature,
    )

    sequence = deepcopy(PASS_SEQUENCE)
    sequence.solve(ip)       

    return (
        draw.__dict__.items() |> map$(t -> (("draw", t[0]), t[1])) |> dict
    ) | (
        FIELDS.items()
        |> starmap$((key, extractor) -> (
                sequence.units 
                |> filter$(u -> isinstance(u, pr.RollPass)) 
                |> map$(u -> ((key, u.label), extractor(u))) 
                |> filter$(t -> t[1] is not None)
            )
        )
        |> flatten
        |> dict            
    )

In [7]:
results = tqdm.tqdm(Pool().imap(worker, draws), total=SAMPLE_COUNT) |> list

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 35.35it/s]


In [8]:
df = results |> enumerate |> dict |> pd.DataFrame |> .T |> .infer_objects()
df

draw                  roll_force                                \
    diameter  temperature             R1             R2             R3   
0   0.050390  1424.351002  398347.148289  230267.925265  424637.189839   
1   0.050772  1429.997863  401235.355794  230970.003851  422274.118749   
2   0.051325  1412.349194  428907.067155  243917.996340  437229.594191   
3   0.049996  1409.882516  402869.428125  233412.129053  433269.310340   
4   0.050787  1423.286940  407514.930390  234163.293715  427085.714654   
..       ...          ...            ...            ...            ...   
95  0.049713  1400.051662  405604.559972  235428.226398  439157.117329   
96  0.050259  1443.972998  378858.744939  220452.799262  410424.599609   
97  0.049412  1416.344465  385015.417093  225317.691618  425999.398782   
98  0.049200  1430.227803  369094.542854  217391.216472  415284.711921   
99  0.049632  1419.146461  387146.817388  225996.920104  424982.833629   

                                                                               \
               R4             R5             R6             R7             R8   
0   180851.502980  256226.926064  138419.671256  260118.562430  118110.394516   
1   179358.109210  254330.416449  137503.648311  258708.956558  117581.358313   
2   183959.180929  259981.951708  140135.594647  262713.584561  119074.199246   
3   184713.867029  261065.700658  140723.855237  263645.466722  119429.904576   
4   181106.223738  256499.294247  138524.970683  260268.983013  118164.253808   
..            ...            ...            ...            ...            ...   
95  187394.248061  264405.935884  142325.255874  266094.059401  120345.969697   
96  175799.878168  249963.539678  135473.638323  255616.618353  116427.883456   
97  183020.160488  259039.278955  139807.949452  262266.279409  118918.990461   
98  179375.926248  254530.972280  137702.637280  259058.364067  117722.447021   
99  182263.482048  258072.925875  139336.999491  261540.852034  118646.533205   

    ...    strain                                                              \
    ...        R5        R6        R7        R8        R9       R10        F1   
0   ...  0.596393  0.499461  0.728196  0.596515  0.458553  0.459857  0.342709   
1   ...  0.596470  0.499475  0.728195  0.596501  0.458543  0.459844  0.342701   
2   ...  0.596622  0.499566  0.728241  0.596553  0.458577  0.459885  0.342724   
3   ...  0.596324  0.499469  0.728213  0.596552  0.458579  0.459892  0.342728   
4   ...  0.596486  0.499495  0.728208  0.596520  0.458556  0.459859  0.342710   
..  ...       ...       ...       ...       ...       ...       ...       ...   
95  ...  0.596270  0.499473  0.728224  0.596578  0.458597  0.459916  0.342741   
96  ...  0.596326  0.499395  0.728156  0.596461  0.458517  0.459812  0.342684   
97  ...  0.596159  0.499398  0.728185  0.596530  0.458566  0.459877  0.342720   
98  ...  0.596075  0.499338  0.728152  0.596491  0.458540  0.459845  0.342703   
99  ...  0.596213  0.499410  0.728186  0.596524  0.458561  0.459870  0.342716   

                                  
          F2        F3        F4  
0   0.346644  0.447918  0.416177  
1   0.346635  0.447914  0.416170  
2   0.346660  0.447925  0.416189  
3   0.346666  0.447928  0.416194  
4   0.346645  0.447919  0.416177  
..       ...       ...       ...  
95  0.346680  0.447934  0.416205  
96  0.346616  0.447906  0.416155  
97  0.346657  0.447924  0.416187  
98  0.346637  0.447915  0.416171  
99  0.346653  0.447922  0.416183  

[100 rows x 100 columns]

In [9]:
df.to_csv(DATA_DIR / OUTPUT_FILENAME)